<a href="https://colab.research.google.com/github/PavankumarUppar/DLL/blob/main/Program6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import string
import numpy as np

# Example text data (replace this with your own text data)
text = "This is an example text for generating text using LSTM in PyTorch. You can replace this with your own text data."

# Preprocess the text and create character mappings
all_characters = string.printable  # All printable ASCII characters
char_to_index = {ch: i for i, ch in enumerate(all_characters)}
index_to_char = {i: ch for i, ch in enumerate(all_characters)}
num_characters = len(all_characters)

# Convert text to a sequence of indices
text_indices = [char_to_index[ch] for ch in text]

# Create input-target pairs
input_indices = text_indices[:-1]
target_indices = text_indices[1:]

# Convert input and target to tensors
input_tensor = torch.tensor(input_indices, dtype=torch.long).unsqueeze(0)
target_tensor = torch.tensor(target_indices, dtype=torch.long).unsqueeze(0)

# Define the LSTM-based text generation model
class LSTMTextGenerator(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMTextGenerator, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.lstm = nn.LSTM(hidden_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden):
        embedded = self.embedding(x)
        output, hidden = self.lstm(embedded, hidden)
        output = self.fc(output)
        return output, hidden

    def init_hidden(self, batch_size):
        return (torch.zeros(self.num_layers, batch_size, self.hidden_size),
                torch.zeros(self.num_layers, batch_size, self.hidden_size))

# Hyperparameters
input_size = num_characters  # Input size is the number of unique characters
hidden_size = 128
num_layers = 2
learning_rate = 0.01
num_epochs = 1000
sequence_length = 100  # Length of sequence used for training

# Instantiate the model, define loss function and optimizer
model = LSTMTextGenerator(input_size, hidden_size, num_layers, input_size)  # Output size = input size
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training the model
for epoch in range(num_epochs):
    model.train()
    hidden = model.init_hidden(1)  # Batch size = 1 for text generation
    total_loss = 0

    optimizer.zero_grad()
    output, hidden = model(input_tensor, hidden)
    loss = criterion(output.squeeze(0), target_tensor.squeeze(0))
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

    if (epoch + 1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss:.4f}')



import random
# Sample text data
text = "This is a sample text. You can replace this with your own text data."
def generate_text(start_letter='T', length=500):
    generated_text = start_letter
    current_letter = start_letter

    for i in range(length):
        next_chars = [text[idx + 1] for idx, char in enumerate(text) if char == current_letter and idx + 1 < len(text)]
        if not next_chars:
            break
        next_letter = random.choice(next_chars)
        generated_text += next_letter
        current_letter = next_letter

    return generated_text

# Generating text using the function
generated_text = generate_text(start_letter='T', length=500)
print("Generated Text:\n", generated_text)



def generate_text(model, start_letter='T', length=500):
    model.eval()
    with torch.no_grad():
        hidden = model.init_hidden(1)
        input_letter = start_letter
        generated_text = input_letter

        for i in range(length):
            input_tensor = torch.tensor([[char_to_index[input_letter]]], dtype=torch.long)
            output, hidden = model(input_tensor, hidden)
            output_probabilities = nn.functional.softmax(output.squeeze(0), dim=0).detach().numpy()
            predicted_index = np.argmax(output_probabilities)
            predicted_char = index_to_char[predicted_index]
            generated_text += predicted_char
            input_letter = predicted_char

    return generated_text

Epoch [100/1000], Loss: 0.0030
Epoch [200/1000], Loss: 0.0011
Epoch [300/1000], Loss: 0.0006
Epoch [400/1000], Loss: 0.0004
Epoch [500/1000], Loss: 0.0003
Epoch [600/1000], Loss: 0.0002
Epoch [700/1000], Loss: 0.0002
Epoch [800/1000], Loss: 0.0001
Epoch [900/1000], Loss: 0.0001
Epoch [1000/1000], Loss: 0.0001
Generated Text:
 Thithis r a. ou dathithis re wis oure is owis wis tampleplextample your wis Yown is cextexteplamplextextaceplexth Youre t ce t. own te th ca wn You owisan t. t da s texthitextatan t t s thisampla s damplatampla. this rexte s text. s is yowisa. repla. you ampla. our texthitan amplat is isa. yown texthitexteple re tacamplacan yowite da damplext. th t ate Yowis own this is is ta. te Yown Yown thith text da You this acaca an t. th t. texte text. yourexte t. own da s ita You is rexta Your ite t. You y
